In [1]:
import numpy as np
import pandas as pd
import nltk
import string as s
from nltk.corpus import stopwords
import seaborn as sns
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
nltk.download('wordnet')

2024-09-11 14:40:40.975096: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-11 14:40:41.152919: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-11 14:40:41.152986: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-11 14:40:41.179980: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-11 14:40:41.261420: I tensorflow/core/platform/cpu_feature_guar

True

In [2]:
# Due to large amount of data , not able to run KNN so. ie limited resorces, more memory required
# Specify the number of rows you want to read
# num_rows_to_read = 60000  # Replace this with the desired number of rows

# Read only the specified number of rows
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.head()

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [4]:
test.head()

,Class Index,Title,Description
0,3,Fears for T N pension after talks,Unions representing workers at Turner Newall...
1,4,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o..."
2,4,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...
3,4,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...
4,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Class Index  120000 non-null  int64 
 1   Title        120000 non-null  object
 2   Description  120000 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.7+ MB


In [6]:
print(train.shape)
print(test.shape)

(120000, 3)
(7600, 3)


In [7]:
train = train.drop_duplicates(subset='Description')
test = test.drop_duplicates(subset='Description')

In [8]:
train_x = train.Description
test_x = test.Description

train_y =  train['Class Index']
test_y = test['Class Index']

In [9]:
train_x = train_x.str.lower()
test_x = test_x.str.lower()

In [10]:
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)

In [11]:
train_x = train_x.apply(striphtml)
test_x = test_x.apply(striphtml)

In [12]:
def remove_url(data):
    return  re.sub(r'\s*(?:https?://)?www\.\S*\.[A-Za-z]{2,5}\s*', ' ', data).strip()

In [13]:
train_x = train_x.apply(remove_url)
test_x = test_x.apply(remove_url)

In [14]:
def word_tok(data):
    tokens = re.findall("[\w']+", data)
    return tokens

In [15]:
train_x = train_x.apply(word_tok)
test_x =test_x.apply(word_tok)

In [16]:
def remove_stopwords(data):
    stopWords = stopwords.words('english')
    new_list = []
    for i in data:
        if i.lower() not in stopWords:
            new_list.append(i)
    return new_list

In [17]:
train_x = train_x.apply(remove_stopwords)
test_x = test_x.apply(remove_stopwords)

In [18]:
def remove_punctuations(data):
    new_list = []
    for i in data:
        for j in s.punctuation:
            i = i.replace(j,'')
        new_list.append(i)
    return new_list

In [19]:
train_x = train_x.apply(remove_punctuations)
test_x = test_x.apply(remove_punctuations)

In [20]:
def remove_number(data):
    no_digit_list = []
    new_list = []

    for i in data:
        for j in s.digits:
            i = i.replace(j,'')
        no_digit_list.append(i)

    for i in no_digit_list:
        if i!='':
            new_list.append(i)
    return new_list

In [21]:
train_x = train_x.apply(remove_number)
test_x = test_x.apply(remove_number)

In [22]:
def stemming(data):
    porter_stemmer = nltk.PorterStemmer()
    roots = [porter_stemmer.stem(i) for i in data]
    return roots

In [23]:
train_x = train_x.apply(stemming)
test_x = test_x.apply(stemming)

In [24]:
def lemmatization(data):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    roots = [lemmatizer.lemmatize(i) for i in data]
    return roots

In [25]:
train_x = train_x.apply(lemmatization)
test_x = test_x.apply(lemmatization)

In [26]:
def remove_extraWords(data):
    extra_words =['href','iii','lt','gt','ii','com','quot']

    new_list = []
    for i in data:
        if i not in extra_words:
            new_list.append(i)
    return new_list

In [27]:
train_x = train_x.apply(remove_extraWords)
test_x = test_x.apply(remove_extraWords)

In [28]:
train_x = [" ".join(map(str, lst)) for lst in train_x]
test_x = [" ".join(map(str, lst)) for lst in test_x]

In [29]:
# Define Single-Layer LSTM Model
def create_single_lstm_model(vocab_size, embedding_dim, max_length, lstm_units):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
    model.add(LSTM(lstm_units))
    model.add(Dropout(0.2))
    model.add(Dense(4, activation='softmax'))  # 4 output classes
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# For the second set, you’ll modify it for two layers:
def create_double_lstm_model(vocab_size, embedding_dim, max_length, lstm_units):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
    model.add(LSTM(lstm_units, return_sequences=True))
    model.add(LSTM(lstm_units))
    model.add(Dropout(0.2))
    model.add(Dense(4, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [48]:
vocab_size = 10000  # Set a reasonable size for the vocabulary
max_length = 200  # You can adjust this based on your data length

In [49]:
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_x)

In [50]:
# Convert the text to sequences
train_sequences = tokenizer.texts_to_sequences(train_x)
test_sequences = tokenizer.texts_to_sequences(test_x)

In [51]:
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post', truncating='post')

In [52]:
from sklearn.model_selection import train_test_split

# Split the training data into training and validation sets (e.g., 80-20 split)
X_train, X_val, y_train, y_val = train_test_split(train_padded, train_y, test_size=0.2, random_state=42)

In [53]:
# Define parameters for single-layer LSTM model
embedding_dim = 10
lstm_units = 8
batch_size = 4

# Create the model
single_lstm_model = create_single_lstm_model(vocab_size, embedding_dim, max_length, lstm_units)

# Train the model
single_lstm_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=batch_size)

Epoch 1/5
23745/23745 [==============================] - 174s 7ms/step - loss: nan - accuracy: 1.0529e-05 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/5
23745/23745 [==============================] - 162s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/5
23745/23745 [==============================] - 182s 8ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/5
23745/23745 [==============================] - 192s 8ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/5
23745/23745 [==============================] - 187s 8ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00


In [54]:
# Define parameters for double-layer LSTM model
embedding_dim = 30
lstm_units = 16
batch_size = 8

# Create the model
double_lstm_model = create_double_lstm_model(vocab_size, embedding_dim, max_length, lstm_units)

# Train the model
double_lstm_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=batch_size)

Epoch 1/5
11873/11873 [==============================] - 175s 15ms/step - loss: nan - accuracy: 1.0529e-05 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/5
11873/11873 [==============================] - 166s 14ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/5
11873/11873 [==============================] - 167s 14ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/5
11873/11873 [==============================] - 160s 13ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/5
11873/11873 [==============================] - 162s 14ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00


In [55]:
# Evaluate the single-layer LSTM model
single_lstm_results = single_lstm_model.evaluate(test_padded, test_y, batch_size=4)
print("Single-layer LSTM test results: ", single_lstm_results)

# Evaluate the double-layer LSTM model
double_lstm_results = double_lstm_model.evaluate(test_padded, test_y, batch_size=8)
print("Double-layer LSTM test results: ", double_lstm_results)

1899/1899 [==============================] - 8s 4ms/step - loss: nan - accuracy: 0.0000e+00
Single-layer LSTM test results:  [nan, 0.0]
950/950 [==============================] - 7s 7ms/step - loss: nan - accuracy: 0.0000e+00
Double-layer LSTM test results:  [nan, 0.0]


In [56]:
# Predict using the single-layer LSTM model
single_predictions = np.argmax(single_lstm_model.predict(test_padded), axis=1)
print("Classification Report for Single-layer LSTM:")
print(classification_report(test_y, single_predictions))

# Predict using the double-layer LSTM model
double_predictions = np.argmax(double_lstm_model.predict(test_padded), axis=1)
print("Classification Report for Double-layer LSTM:")
print(classification_report(test_y, double_predictions))

238/238 [==============================] - 1s 4ms/step
Classification Report for Single-layer LSTM:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00    1900.0
           2       0.00      0.00      0.00    1899.0
           3       0.00      0.00      0.00    1896.0
           4       0.00      0.00      0.00    1899.0

    accuracy                           0.00    7594.0
   macro avg       0.00      0.00      0.00    7594.0
weighted avg       0.00      0.00      0.00    7594.0



/home/rimuru/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/rimuru/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/rimuru/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/rimuru/.local/lib/python

238/238 [==============================] - 2s 5ms/step
Classification Report for Double-layer LSTM:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00    1900.0
           2       0.00      0.00      0.00    1899.0
           3       0.00      0.00      0.00    1896.0
           4       0.00      0.00      0.00    1899.0

    accuracy                           0.00    7594.0
   macro avg       0.00      0.00      0.00    7594.0
weighted avg       0.00      0.00      0.00    7594.0



/home/rimuru/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/rimuru/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/rimuru/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/rimuru/.local/lib/python